# Neural Networks

We are going to see an example of what we can do with the functions described in the [Linear Algebra tutorial](https://ironarray.io/docs/html/tutorials/linear-algebra.html). We are going to implement a simple neural network using these functions and some other *ironArray* modules like expressions or random generators.

First of all, we have to generate the dataset that we use in our Neural Network.

In [17]:
import iarray as ia
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

coord, cl = make_classification(20000, 1000)
X, Xt, y, yt = train_test_split(coord, cl, test_size=0.3)

training_inputs = ia.numpy2iarray(X.astype(np.float64))
training_labels = ia.numpy2iarray(y.astype(np.float64).reshape(-1, 1))
inputs = ia.numpy2iarray(Xt.astype(np.float64))

Now, we create a class implementing a neural network with a hidden layer.

In [18]:
class NN(object):
    
    def __init__(self, input_layer, hidden_layer, output_layer, random_seed=1):
        ia.set_config_defaults(seed=random_seed)
        self.l1_weights = ia.random.standard_normal((input_layer, hidden_layer))
        self.l2_weights = ia.random.standard_normal((hidden_layer, output_layer))

    def _sigmoid(self, x):
        return ia.expr_from_string("1 / (1 + exp(-x))", {"x": x}).eval()

    def _sigmoid_prime(self, x):
        return ia.expr_from_string("x * (1 - x)", {"x": x}).eval()
    
    def feed_forward(self, X):
        l1_output = ia.matmul(X, self.l1_weights)
        l1_output = self._sigmoid(l1_output)

        l2_output = ia.matmul(l1_output, self.l2_weights)
        l2_output = self._sigmoid(l2_output)

        return l1_output, l2_output
    
    def backpropagation(self, l1, l2, y):
        
        l2_error = (y - l2).eval()
        
        l2_delta = (l2_error * self._sigmoid_prime(l2)).eval()

        l1_error = ia.matmul(l2_delta, self.l2_weights.T)

        l1_delta = (l1_error * self._sigmoid_prime(l1)).eval()
        
        return l2_error, l1_delta, l2_delta

    def update_weights(self, X, l1, l1_delta, l2_delta, alpha=0.01):
        expr = f"w + {alpha} * d"
        d2 = ia.matmul(l1.T, l2_delta)
        d1 = ia.matmul(X.T, l1_delta)

        self.l2_weights = ia.expr_from_string(expr, {"w": self.l2_weights, "d": d2}).eval()
        self.l1_weights = ia.expr_from_string(expr, {"w": self.l1_weights, "d": d1}).eval()

    def predict(self, X):
        _, l2 = self.feed_forward(X)
        return l2

    def train(self, X, y, threshold=400, alpha=0.01):
        for j in range(threshold + 1):
            l1, l2 = self.feed_forward(X)
            l2_error, l1_delta, l2_delta = self.backpropagation(l1, l2, y)
            self.update_weights(X, l1, l1_delta, l2_delta, alpha=alpha)
            if(j % 100 == 0):
                train_error = ia.mean(ia.abs(l2_error).eval())
                print("epoch {:5} ".format(j),end='-')
                print(' error: {:0.4f} '.format(train_error))

Once the NN is created, we create a NN and train it with our dataset.

In [ ]:
nn = NN(X.shape[1], 5, 1)

nn.train(training_inputs, training_labels, threshold=500, alpha=0.1)

epoch     0 - error: 0.4954 
epoch   100 - error: 0.0725 
epoch   200 - error: 0.0465 
epoch   300 - error: 0.0441 
epoch   400 - error: 0.0434 


Finally, we predict the test part of our dataset and print the accuracy score :)

In [ ]:
labels = nn.predict(inputs)
labels = np.ravel((labels.data > 0.5).astype(int))
accuracy_score(labels, yt)
